In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss

In [16]:
%run "./scripts/start.py"
df_full, df_train, df_test = IO_full()
print(len(df_train))
df_full, df_test = None, None # This notebook is for validation, no test set needed
df_train = make_simple_features(df_train)
simple_feats = ["bathrooms", "bedrooms", "price",        
                "num_photos", "num_features", "num_description_words",                    
                "created_month", "created_day", "created_hour"
            ]

49352


In [21]:
#import os
#cwd = os.getcwd()

In [18]:
cluster_feat = pd.read_csv("./features/mk/train_kmean_ft.csv")
assert len(cluster_feat) == 49352

In [19]:
df_train = pd.merge(df_train, cluster_feat, left_index=True, right_index=True)

## It was faster to just use your function that defines folds, than to set up a function to read files and assemble them in the right order. Results are the same and should be consistent, because random state is defined.

In [31]:
def make_folds(df_train, df_test):
    from sklearn.model_selection import StratifiedKFold
    col_feats = [col for col in df_train if col != "interest_level"]
    X = df_train[col_feats]
    target_num_map={"high":0, "medium":1, "low":2}
    y = df_train["interest_level"].apply(lambda x: target_num_map[x])
    skf = StratifiedKFold(n_splits=5, random_state = 0)
    skf.get_n_splits(X, y)
    return skf

In [59]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

clf=RFC(n_estimators=100, random_state=42)

# define CV split functionaly
cv_sets_mk = make_folds(df_train,df_test)
#train_index, valid_index = IO_SkF()

features = simple_feats + ["cluster_35"]

target_num_map={"high":0, "medium":1, "low":2}
df_train["interest_level_num"]=df_train["interest_level"].apply(lambda x: target_num_map[x]).values

#print("Referring to both the seen data and the train fold as 'train' is confusing so tell me if you have better terminology")
#X_train, y_train =df_train.loc[train_index, features].copy(deep=True), df_train.loc[train_index, "interest_level_num"].copy(deep=True) 
#X_val, y_val = df_train.loc[valid_index, features].copy(deep=True), df_train.loc[valid_index, "interest_level_num"].copy(deep=True)

#for col in features:
#    assert len(X_train[X_train[col].isnull()]) == 0   


#the scoring here is defined as negative log-loss metric, don't know why, but easy to change - just multipy by -1
cross_val_scores = cross_val_score(clf, df_train[features], df_train["interest_level_num"], cv=cv_sets_mk, scoring='neg_log_loss')
print(-cross_val_scores)
print("average error", np.average(-cross_val_scores))
#y_val_pred = cross_val_predict(clf,df_train[features])

#Unfortunately we can't get to complete array of predictions from CV - this is know issue with multiclass regression problems
#cross_val_predict returns an array of the same size as `y` where each entry
# is a prediction obtained by cross validation:
#y_val_pred = cross_val_predict(clf, df_train[features], df_train["interest_level_num"], cv=cv_sets_mk, n_jobs = -1)

[ 0.68104073  0.67163068  0.66798303  0.67041652  0.66906537]
average error 0.672027262563


In [60]:
# this is the actual model training for submission
clf_full = clf.fit(df_train[features], df_train["interest_level_num"])